In [ ]:
import tweepy           
import pandas as pd     
import numpy as np      
import sys
import datetime
import csv

# For draft and visualization
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
consumer_key = 'SY1TACPOFQ1CBVmO4OQhw9lpX'
consumer_secret = 'fx4fFoA71CGdtwYUmjr5mnXUAIkHbmOvmw91h5WSi20zMirnYp'
access_token = '1108216575935430656-mbqcloTcgc3WFDMvEvwL1WfT34GhQR'
access_token_secret = 'sz4rlbfZiLZo1612LzAss7VwdGUIP7zsFS7B5EcuAbzNt'
def twitter_setup():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    return api

## Test

In [ ]:
# create a extractor object
extractor = twitter_setup()

# create a tweet list
tweets = extractor.user_timeline(screen_name="realDonaldTrump", count=200)
print("Number of tweets extracted: {}.\n".format(len(tweets)))

# Print last 5 tweets
print("5 recent tweets:\n")
for tweet in tweets[:5]:
    print(tweet.text)
print()
# Create a Pandas dataframe
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
display(data.head(10))

print(tweets[0].id)
print(tweets[0].created_at)
print(tweets[0].source)
print(tweets[0].favorite_count)
print(tweets[0].retweet_count)
print(tweets[0].geo)
print(tweets[0].coordinates)
print(tweets[0].entities)
print(tweets[0].__hash__)

## Internal methods

In [ ]:
# internal methods in a single tweet object
print(dir(tweets[0]))

In [ ]:
print(tweets[0].id)
print(tweets[0].created_at)
print(tweets[0].source)
print(tweets[0].favorite_count)
print(tweets[0].retweet_count)
print(tweets[0].geo)
print(tweets[0].coordinates)
print(tweets[0].entities)
print(tweets[0].__hash__)

## Set time period

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
username = sys.argv[1]

# Set a timeline
startDate = datetime.datetime(2019, 3, 1, 0, 0, 0)
endDate =   datetime.datetime(2019, 3, 2, 0, 0, 0)

tweets = []
toptweets = []

users = []
with open('VicGov.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter = ',')
    for row in readCSV:
        users.append(row[0])

## Timeline

In [ ]:
favs = []
rts = []
for i in range(1, len(users)):
    # Extract all the tweets of a account between two certain times
    tmpTweets = api.user_timeline(users[i])
    tweets = []
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)

    while (tmpTweets[-1].created_at > startDate):
        tmpTweets = api.user_timeline(users[i], max_id = tmpTweets[-1].id)
        for tweet in tmpTweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
                tweets.append(tweet)
    Likes = []
    RTs = []

    if(tweets):
        for tweet in tweets:
            Likes.append(tweet.favorite_count)
            RTs.append(tweet.retweet_count)


    if(Likes):
        favs.append(max(Likes))
        i = Likes.index(max(Likes))
        toptweets.append(tweets[i])
    else:
        favs.append('0')

    if(RTs):
        rts.append(max(RTs))
    else:
        rts.append('0')

print(users[1:], len(users[1:]))
print(favs, len(favs))
print(rts, len(rts))

data = pd.DataFrame(data=users[1:], columns=['User'])
data['fav']  = np.array(favs)
data['rt']  = np.array(rts)
display(data)

In [ ]:
data = pd.DataFrame(data=[toptweet.text for toptweet in toptweets], columns=['Tweets'])
data['len']  = np.array([len(toptweet.text) for toptweet in toptweets])
data['ID']   = np.array([toptweet.id for toptweet in toptweets])
data['Date'] = np.array([toptweet.created_at for toptweet in toptweets])
data['Source'] = np.array([toptweet.source for toptweet in toptweets])
data['Likes']  = np.array([toptweet.favorite_count for toptweet in toptweets])
data['RTs']    = np.array([toptweet.retweet_count for toptweet in toptweets])

In [ ]:
tfav = pd.Series(data=data['Likes'].values, index=data['Date'])
tret = pd.Series(data=data['RTs'].values, index=data['Date'])
tfav.plot(figsize=(40,4), label="Likes", legend=True)
tret.plot(figsize=(40,4), label="Retweets", legend=True);